# Street End Parameter Tuning

This notebook helps tune parameters for the street end finder algorithm and visualize results.

In [ ]:
import sys
sys.path.append('..')

from main import StreetEndFinder
from render_map import StreetEndRenderer
from global_analysis import GlobalCityAnalyzer
import folium
from IPython.display import display, HTML, IFrame
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

## Parameter Configuration

Adjust these parameters to tune the algorithm

In [ ]:
# Test parameters
params = {
    'city': 'Skokie, USA',
    'threshold_distance': 10,     # meters from water
    'buffer_distance': 100,       # meters around water to search for streets
    'min_point_spacing': 25,      # minimum meters between points
    'find_street_ends': True,     # whether to find street ends
    'enrich_data': True,         # whether to enrich with additional data
    'update_existing': True      # whether to update existing analysis
}

# Display current parameters
pd.DataFrame([params]).T

## Run Single City Analysis

In [ ]:
def analyze_single_city(params):
    """Run analysis for a single city with given parameters"""
    # Stage 1: Find street ends
    finder = StreetEndFinder(
        params['city'],
        threshold_distance=params['threshold_distance']
    )
    water_features, near_water = finder.process()
    
    # Create initial visualization
    renderer = StreetEndRenderer(
        'street_ends_near_river.geojson',
        params['city'],
        threshold_distance=params['threshold_distance']
    )
    renderer.render('temp_map.html')
    
    return water_features, near_water, renderer

# Run analysis
water_features, near_water, renderer = analyze_single_city(params)

# Display the map in the notebook
display(IFrame(src='temp_map.html', width='100%', height=600))

## Analysis Statistics

In [ ]:
def get_statistics(near_water, water_features):
    """Calculate and display statistics about the results"""
    stats = {
        'Total Water Features': len(water_features) if water_features is not None else 0,
        'Total Street Ends Near Water': len(near_water),
        'Average Distance to Water': np.mean([p['properties']['distance_to_water'] for p in near_water]) if near_water else 0,
        'Min Distance to Water': min([p['properties']['distance_to_water'] for p in near_water]) if near_water else 0,
        'Max Distance to Water': max([p['properties']['distance_to_water'] for p in near_water]) if near_water else 0
    }
    return pd.DataFrame([stats]).T

get_statistics(near_water, water_features)

## Distance Distribution

In [ ]:
def plot_distance_distribution(near_water):
    """Plot distribution of distances to water"""
    distances = [point['properties']['distance_to_water'] for point in near_water]
    
    plt.figure(figsize=(10, 6))
    sns.histplot(distances, bins=20)
    plt.title('Distribution of Distances to Water')
    plt.xlabel('Distance (meters)')
    plt.ylabel('Count')
    plt.show()

plot_distance_distribution(near_water)

## Multi-City Analysis

In [ ]:
def run_multi_city_analysis(cities, params):
    """Run analysis for multiple cities"""
    # Stage 1: Find street ends
    analyzer_stage1 = GlobalCityAnalyzer(
        find_street_ends=params['find_street_ends'],
        enrich_data=False,
        update_existing=params['update_existing']
    )
    analyzer_stage1.run_global_analysis_stage1(target_cities=cities)
    
    # Stage 2: Enrich data
    analyzer_stage2 = GlobalCityAnalyzer(
        find_street_ends=False,
        enrich_data=params['enrich_data'],
        update_existing=params['update_existing']
    )
    results = analyzer_stage2.run_global_analysis_stage2(target_cities=cities)
    
    # Stage 3: Generate report
    analyzer_stage3 = GlobalCityAnalyzer()
    analyzer_stage3.run_global_analysis_stage3()
    
    return results

# Example multi-city analysis
cities_to_analyze = ['Skokie', 'Chicago']
results = run_multi_city_analysis(cities_to_analyze, params)

# Display the global report
display(IFrame(src='global_report.html', width='100%', height=800))

## Compare Cities

In [ ]:
def compare_cities(results):
    """Create comparative visualizations of cities"""
    # Convert results to DataFrame
    df = pd.DataFrame([
        {
            'city': r['properties']['city_name'],
            'street_ends': r['properties']['total_street_ends'],
            'population': r['properties']['population']
        } for r in results
    ])
    
    # Create comparison plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Street ends by city
    sns.barplot(data=df, x='city', y='street_ends', ax=ax1)
    ax1.set_title('Street Ends by City')
    ax1.tick_params(axis='x', rotation=45)
    
    # Street ends per capita
    df['ends_per_capita'] = df['street_ends'] / df['population'] * 100000
    sns.barplot(data=df, x='city', y='ends_per_capita', ax=ax2)
    ax2.set_title('Street Ends per 100k Population')
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return df

comparison_df = compare_cities(results)
display(comparison_df)